## Long Short-Term Memory 

In this assignment, we will learn about LSTM models. We will create an LSTM model for time series prediction.

In [1]:
import numpy as np
import os
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten
from tensorflow.keras.utils import to_categorical

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Below is a function for loading time series data collected by sensors. There are 9 different files, We have data about body acceleration, body gyro, and total acceleration for the x, y, and z axis

In [4]:
def load_func(path, file_ind=False):
    data_list = []
    if file_ind:
        filenames = [path]
    else:
        files = os.listdir(path)
        filenames = [path + '/' + f for f in files]
    for f in filenames:
        dataframe = pd.read_csv(f, header=None, delim_whitespace=True)
        data_list.append(dataframe.values)
    if len(data_list) > 1:
        return np.dstack(data_list)
    else:
        return data_list[0]

In [13]:
PATH = '/content/drive/MyDrive/Data/UCI HAR Dataset/'

In [14]:

os.listdir(f'{PATH}train/Inertial Signals')

FileNotFoundError: ignored

In [15]:
X_train = load_func("./train/Inertial Signals")
X_test = load_func("./test/Inertial Signals")
y_train_cat = load_func('./train/y_train.txt', True)
y_test_cat = load_func('./test/y_test.txt', True)

FileNotFoundError: ignored

Print the dimensions of both the predictor variables and the target.

In [ ]:
# Answer below:

print(f'X train shape: {X_train.shape}')
print(f'Y train shape: {y_train_cat.shape}')
print(f'X test shape: {X_test.shape}')
print(f'Y test shape: {y_test_cat.shape}')

The target variable is categorical. One hot encode the target variable.

In [ ]:
# Answer below:


from sklearn.preprocessing import OneHotEncoder

#enc = OneHotEncoder(handle_unknown='ignore') 
#y_train = enc.fit_transform(y_train_cat).toarray() 
#y_test = enc.transform(y_test_cat).toarray()

y_train = to_categorical(y_train_cat - 1, 6)
y_test = to_categorical(y_test_cat - 1, 6)

y_train.shape

Create a model containing an LSTM layer with unit size 100, and input shape that is the tuple containing the number of columns in X and the number of files in X.

The next layer is a dropout layer. Choose 0.5 for the proportion. Then add a dense layer of unit size 100 and finally an output dense layer. 

In [ ]:
# Answer below:


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# instantiate model
model = Sequential()
# LSTM layer
model.add(LSTM(100, input_shape=(X_train.shape[1],X_train.shape[2])))
# dropout layer
model.add(Dropout(0.5))
# dense layer
model.add(Dense(100, activation='relu'))
# dense output layer
model.add(Dense(y_train.shape[1], activation='softmax'))

Print the model summary to ensure you have the correct number of parameters.

In [ ]:
# Answer below:

model.summary()

Compile and fit the model. Select the appropriate activation, loss, and optimizer.

Run the model for 10 epochs with a batch size of 80.

In [ ]:
# Answer below:

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=80)

Print the confusion matrix for the test data.

In [ ]:
# Answer below:

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    Given a scikit-learn confusion matrix (CM), make a nice plot.

    Arguments
    ---------
    cm:           Confusion matrix from sklearn.metrics.confusion_matrix

    target_names: Given classification classes, such as [0, 1, 2]
                  The class names, for example, ['high', 'medium', 'low']

    title:        The text to display at the top of the matrix

    cmap:         The gradient of the values displayed from matplotlib.pyplot.cm
                  See http://matplotlib.org/examples/color/colormaps_reference.html
                  `plt.get_cmap('jet')` or `plt.cm.Blues`

    normalize:    If `False`, plot the raw numbers
                  If `True`, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # Confusion matrix created by
                                                              # `sklearn.metrics.confusion_matrix`
                          normalize    = True,                # Show proportions
                          target_names = y_labels_vals,       # List of names of the classes
                          title        = best_estimator_name) # Title of graph

    Citation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('Predicted label')
    plt.xlabel('True label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
y_test_pred = np.argmax(model.predict(X_test), axis=-1)
y_test_pred

y_test_pred[1]

In [ ]:

from sklearn.metrics import  classification_report, confusion_matrix

# plot model confusion matrix
confusion = confusion_matrix(y_test_cat-1, y_test_pred)
plot_confusion_matrix(cm=confusion, target_names=[1,2,3,4,5,6],title = 'Model Confusion Matrix',normalize=False)

In [ ]:
:
# model classifcation report
print(classification_report(y_test_cat-1, y_test_pred))